In [1]:
from src.models import train_model
import pandas as pd
import mlflow

import os

os.environ["AWS_PROFILE"] = "default"
os.environ["AWS_DEFAULT_REGION"] = "ap-southeast-1"

df = pd.read_parquet("../data/final/new_df.parquet")
dataset = df[
    [
        "is_nsfw",
        "display_state",
        "flags.os_hidden",
        "flags.os_is_nsfw",
        "flags.code_verified",
        "flags.functions_threshold",
        "flags.approval_threshold",
        "flags.bytecode_threshold",
        "flags.did_self_destruct",
        "flags.is_whitelisted_proxy",
        "display_stats.floor_price",
        "display_stats.supply",
        "display_stats.holders",
        "display_stats.one_day_volume",
        "display_stats.seven_day_volume",
        "display_stats.thirty_day_volume",
        "display_stats.total_volume",
        "external.telegram_total_member",
        "external.has_instagram",
        "external.discord_total_member",
        "external.has_twitter",
        "has_banner_image_url",
        "has_description",
        "has_collection_image_url",
        "has_display_name",
    ]
]


In [23]:
# url = 'http://localhost:5000'
# mlflow.set_tracking_uri(url)
experiment = mlflow.get_experiment_by_name("pycaret")
experiment


<Experiment: artifact_location='file:///C:/Users/Victor/Documents/code%20projects/python/interview%20%26%20jobs/DTTD/FITE4801/notebooks/mlruns/1', creation_time=1681289320171, experiment_id='1', last_update_time=1681289320171, lifecycle_stage='active', name='pycaret', tags={}>

In [21]:
runs = mlflow.search_runs(experiment.experiment_id)

# Filter runs by a specific metric or any other criteria
# For example, find the run with the highest accuracy
best_run = runs.loc[runs["metrics.AUC"].idxmax()]

# Get the run ID and artifact URI of the best run
best_run_id = best_run.run_id
best_run_name = best_run["tags.mlflow.runName"].replace(" ", "_")
artifact_uri = best_run.artifact_uri
registered_model_name = "best-model-2023-04-01"


In [30]:
dict(best_run)

{'run_id': '9e9f5c7485a840dabaad9cfe0cbe50ee',
 'experiment_id': '1',
 'status': 'FINISHED',
 'artifact_uri': 'file:///C:/Users/Victor/Documents/code%20projects/python/interview%20%26%20jobs/DTTD/FITE4801/notebooks/mlruns/1/9e9f5c7485a840dabaad9cfe0cbe50ee/artifacts',
 'start_time': Timestamp('2023-04-13 06:46:02.091000+0000', tz='UTC'),
 'end_time': Timestamp('2023-04-13 06:46:03.021000+0000', tz='UTC'),
 'metrics.Kappa': 0.9633,
 'metrics.F1': 0.972,
 'metrics.AUC': 0.9976,
 'metrics.Recall': 0.9723,
 'metrics.Prec': 0.9719,
 'metrics.MCC': 0.9633,
 'metrics.TT': 0.144,
 'metrics.Accuracy': 0.9723,
 'params.tol': None,
 'params.store_covariance': None,
 'params.priors': None,
 'params.reg_param': None,
 'params.var_smoothing': None,
 'params.strategy': None,
 'params.random_state': '123',
 'params.constant': None,
 'params.epsilon': None,
 'params.alpha': None,
 'params.warm_start': None,
 'params.l_ratio': None,
 'params.verbose': None,
 'params.power_t': None,
 'params.learning_rat

In [12]:
# Define the path to the model artifact (logged as 'model' in the example)
model_path = os.path.join(artifact_uri.replace(r"file://", ""), "model").lstrip("/")
model_path


'C:/Users/Victor/Documents/code%20projects/python/interview%20%26%20jobs/DTTD/FITE4801/notebooks/mlruns/1/9e9f5c7485a840dabaad9cfe0cbe50ee/artifacts\\model'

In [32]:
mlflow.register_model(
    model_uri=model_path,
    name=registered_model_name,
    tags=dict(best_run),
)


Successfully registered model 'best-model-2023-04-01'.
2023/04/17 10:03:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: best-model-2023-04-01, version 1
Created version '1' of model 'best-model-2023-04-01'.


<ModelVersion: creation_timestamp=1681697000553, current_stage='None', description=None, last_updated_timestamp=1681697000553, name='best-model-2023-04-01', run_id=None, run_link=None, source='C:/Users/Victor/Documents/code%20projects/python/interview%20%26%20jobs/DTTD/FITE4801/notebooks/mlruns/1/9e9f5c7485a840dabaad9cfe0cbe50ee/artifacts\\model', status='READY', status_message=None, tags={'artifact_uri': 'file:///C:/Users/Victor/Documents/code%20projects/python/interview%20%26%20jobs/DTTD/FITE4801/notebooks/mlruns/1/9e9f5c7485a840dabaad9cfe0cbe50ee/artifacts',
 'end_time': '2023-04-13 06:46:03.021000+00:00',
 'experiment_id': '1',
 'metrics.AUC': '0.9976',
 'metrics.Accuracy': '0.9723',
 'metrics.F1': '0.972',
 'metrics.Kappa': '0.9633',
 'metrics.MCC': '0.9633',
 'metrics.Prec': '0.9719',
 'metrics.Recall': '0.9723',
 'metrics.TT': '0.144',
 'params.C': 'None',
 'params.CPU Jobs': 'None',
 'params.Categorical iterative imputer': 'None',
 'params.Experiment Name': 'None',
 'params.Fol

In [3]:
train_model.train_model(dataset, experiment_name="pycaret1", train_size=0.1)


Data for Modeling: (44032, 25)


ProfileNotFound: The config profile (default) could not be found

In [4]:
!mlflow ui

^C


In [8]:
from pycaret.classification import load_model

logged_model = "mlruns/1/9e9f5c7485a840dabaad9cfe0cbe50ee/artifacts/model/model"
pipeline = load_model(logged_model)


Transformation Pipeline and Model Successfully Loaded


In [10]:
pipeline.predict(dataset.drop("display_state", axis=1))


array([1, 5, 0, ..., 4, 2, 3], dtype=int64)